In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-qk6jha4a
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-qk6jha4a
     |████████████████████████████████| 344 kB 4.1 MB/s 
     |████████████████████████████████| 47.3 MB 1.1 MB/s 
     |████████████████████████████████| 4.9 MB 22.4 MB/s 
     |████████████████████████████████| 1.2 MB 69.6 MB/s 
     |████████████████████████████████| 3.4 MB 78.1 MB/s 
     |████████████████████████████████| 895 kB 76.7 MB/s 
     |████████████████████████████████| 596 kB 60.9 MB/s 
     |████████████████████████████████| 61 kB 626 kB/s 
     |████████████████████████████████| 3.3 MB 73.3 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=f8582acd7ca54059064e2df5d7a40a0503699e598fd93f65c58e851bbd276266
  Stored in directory: /tmp/pip-ephem-wheel-cache-jzu07raa/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd

In [4]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [5]:
!pip install mxnet
!pip install gluonnlp padas tqdm
!pip install sentencepiece
!pip install tansformers

ERROR: Could not find a version that satisfies the requirement padas (from versions: none)
ERROR: No matching distribution found for padas
ERROR: Could not find a version that satisfies the requirement tansformers (from versions: none)
ERROR: No matching distribution found for tansformers


In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [7]:
# GPU 사용

use_cuda = True
device = torch.device("cuda:0")

In [8]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [9]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [10]:
#데이터 셋 불러오기
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/jjal_project/감성대화말뭉치.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
# 데이터 셋 확인
data = data.fillna('')
data = data[['감정_소분류', '사람문장1', '사람문장2', '사람문장3', '사람문장4']].iloc[:46009, :]
data

,감정_소분류,사람문장1,사람문장2,사람문장3,사람문장4
0,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아 지금 정말 신이 나.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,
1,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,
2,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.
3,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,회사생활을 열심히 해서 계속 월급을 받을거야!,
4,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,,
...,...,...,...,...,...
46004,느긋,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다.,그래서 마음의 안정을 가질 수 있었던 것 같아.,돈을 함부로 쓰지 말고 지금 현재에 만족하면서 지내야지.,
46005,회의적인,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 ...,이대로 혹시 더 심해져서 죽게 되는 건 아닐까?,의사 선생님에게 악화될 확률은 얼마나 되는지 정확히 물어봐야겠어.,
46006,좌절한,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야.,내가 게을러서 다른 방법을 아직 생각하진 못했어. 댄스 동호회는 좀 괜찮은 것 같은...,,
46007,안달하는,은행 대출이 막혀서 생활비를 구할 수가 없어. 이제 어떻게 살아야 하나 막막해.,당장 내일 밥은 어떡하고 내 병원비는 어쩌지.,어떻게 해야 할 지 모르겠어. 눈 앞이 캄캄해.,


In [12]:
from pandas.core.common import random_state
from sklearn.preprocessing import LabelEncoder
# train data와 label 분류
sentence = data['사람문장1'] +' '+ data['사람문장2'] +' '+ data['사람문장3'] +' '+ data['사람문장4']
emotion = data['감정_소분류']
le = LabelEncoder()
emotion = le.fit_transform(emotion)

# label 비율별로 train/test split
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1004)

for train_idx, test_idx in split.split(sentence, emotion):
    X_train = sentence[train_idx]
    X_test = sentence[test_idx]
    y_train = emotion[train_idx]
    y_test = emotion[test_idx]

In [13]:
# dataset으로 결합
dataset_train = []
for x, y in zip(X_train, y_train):
    data = []
    data.append(x)
    data.append(y)
    dataset_train.append(data)

dataset_test = []
for x, y in zip(X_test, y_test):
    data = []
    data.append(x)
    data.append(y)
    dataset_test.append(data)

In [14]:
# 입력 데이터 생성

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
# 하이퍼파라미터 설정
max_len = 100
batch_size = 32
warmup_ratio = 0.2
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  3e-5

In [16]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [18]:
# KoBERT 학습 모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=58,
                 dr_rate=0.1,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.1).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [20]:
# 훈련전 gpu cache 정리
torch.cuda.empty_cache()
import gc
gc.collect()

89

In [21]:
# 모델 이어서 훈련할 때 : 모델 불러오기

# checkpoint = torch.load('/content/drive/MyDrive/말뭉치/.pt')
# model.load_state_dict(checkpoint['state'])
# optimizer.load_state_dict(checkpoint['optimizer'])

In [22]:
from torch.autograd import Variable

In [26]:
#모델 학습

former_acc = 0.0
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

# test_acc가 커질 때만 모델 저장
    if test_acc / (batch_id+1) > former_acc:

        torch.save({
            'state' : model.state_dict(),
            'optimizer' : optimizer.state_dict(),
        }, '/content/drive/MyDrive/jjal_project/model{}.pt'.format(test_acc / (batch_id+1)))

        former_acc = test_acc / (batch_id+1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1151 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8486987352371216 train acc 0.5625
epoch 1 batch id 201 loss 1.8028874397277832 train acc 0.47310322523117065
epoch 1 batch id 401 loss 2.1443369388580322 train acc 0.48223191499710083
epoch 1 batch id 601 loss 2.3773491382598877 train acc 0.4811772108078003
epoch 1 batch id 801 loss 2.266655921936035 train acc 0.48388731479644775
epoch 1 batch id 1001 loss 2.14988374710083 train acc 0.48907342553138733
epoch 1 train acc 0.49042367935180664


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/288 [00:00<?, ?it/s]

...

  0%|          | 0/288 [00:00<?, ?it/s]

epoch 8 test acc 0.498601496219635


  0%|          | 0/1151 [00:00<?, ?it/s]

epoch 9 batch id 1 loss 0.18599723279476166 train acc 0.96875


KeyboardInterrupt: ignored

In [27]:
small_model = torch.load('/content/drive/MyDrive/jjal_project/model0.498601496219635.pt')
small_model = model.cuda()

In [28]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=4)
    
    small_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = small_model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            for i in range(58):
                if np.argmax(logits) == i:
                    test_eval.append(le.inverse_transform(np.array([i])))

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
 

In [29]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 이러다 다 죽어
['>> 입력하신 내용에서 좌절한 느껴집니다.']


하고싶은 말을 입력해주세요 : 벌레 너무 싫다
['>> 입력하신 내용에서 혐오스러운 느껴집니다.']


하고싶은 말을 입력해주세요 : 나 어떻게 해...?
['>> 입력하신 내용에서 충격 받은 느껴집니다.']


하고싶은 말을 입력해주세요 : 진짜 짜증나서 미쳐버릴 것 같아
['>> 입력하신 내용에서 짜증내는 느껴집니다.']


하고싶은 말을 입력해주세요 : 당황스럽네...
['>> 입력하신 내용에서 혼란스러운 느껴집니다.']


하고싶은 말을 입력해주세요 : 리얼 죽어버릴까?
['>> 입력하신 내용에서 악의적인 느껴집니다.']


하고싶은 말을 입력해주세요 : 죽여버릴까?
['>> 입력하신 내용에서 악의적인 느껴집니다.']


하고싶은 말을 입력해주세요 : 죽자 그냥
['>> 입력하신 내용에서 악의적인 느껴집니다.']


하고싶은 말을 입력해주세요 : 지랄하고 자빠졌네
['>> 입력하신 내용에서 신이 난 느껴집니다.']


하고싶은 말을 입력해주세요 : 기분이 너무 좋다!
['>> 입력하신 내용에서 신이 난 느껴집니다.']


하고싶은 말을 입력해주세요 : 혼란스럽다
['>> 입력하신 내용에서 혼란스러운 느껴집니다.']


하고싶은 말을 입력해주세요 : 여기 진짜 맛있었어
['>> 입력하신 내용에서 신이 난 느껴집니다.']


하고싶은 말을 입력해주세요 : 너무 만족스럽다
['>> 입력하신 내용에서 만족스러운 느껴집니다.']


하고싶은 말을 입력해주세요 : 난 너무 멍청한가봐ㅜㅜ
['>> 입력하신 내용에서 고립된 느껴집니다.']


하고싶은 말을 입력해주세요 : 진짜 싫어!
['>> 입력하신 내용에서 혐오스러운 느껴집니다.']


하고싶은 말을 입력해주세요 : 1
['>> 입력하신 내용에서 부끄러운 느껴집니다.']


하고싶은 말을 입력해주세요 : 부끄러웡
['>> 입력하신 내용에서 부끄러운 느껴집니다.']




KeyboardInterrupt: ignored